In [1]:
import numpy as np

In [2]:
all_checks =[(0, -1), (0, +1)] + [(-1, c) for c in range(-1,2)] + [(+1, c) for c in range(-1,2)]
assert(len(set(all_checks)) == 8)

#check_list = ['North', 'South', 'West', 'East']

check_dict = {
    'North': [(-1, c) for c in range(-1,2)],
    'South': [(+1, c) for c in range(-1,2)],
    'West': [(0,-1), (-1,-1), (+1,-1)],
    'East': [(0,+1), (-1,+1), (+1,+1)],
}

move_dict = {
    'North': (-1,0),
    'South': (+1,0),
    'West': (0,-1),
    'East': (0,+1),  
}

# very lazy....def a cleaner way to do this!
check_list = [
    ['North', 'South', 'West', 'East'],
    ['South', 'West', 'East', 'North'],
    ['West', 'East', 'North', 'South'],
    ['East', 'North', 'South', 'West'],
]


In [3]:
def full_scan(matrix, all_checks):
    """
    For each location of elf determine total neighbors. If 0 neighbors around
    then do not include in eligible list.
    
    """
    elves = np.argwhere(matrix == '#')
    passed_elves = []
    
    for (r1,c1) in elves:
        neighbor_count = 0
        for (r_check, c_check) in all_checks:
            if matrix[(r1 + r_check, c1 + c_check)] == '#':
                neighbor_count += 1
        if neighbor_count > 0:
            passed_elves.append((r1, c1))
    return passed_elves

def move_proposal(matrix, elves, check_dict, check_list):
    """
    Return an index of values that pass checks
    
    Assume check_list is properly ordered 
    """
    passed_elves = []
    for (r1,c1) in elves:
        #print(f"Checking elf: {r1,c1}")
        add_flag = False
        
        dir_check = 0
        while (add_flag == False):# or (dir_check < 3):
            direction = check_list[dir_check]
            #print(f"Checking direction {direction}")
            fail_flag = False
            for (r_check, c_check) in check_dict[direction]:
                #print(f"Checking {direction, r_check, c_check}")
                if matrix[(r1 + r_check, c1 + c_check)] == '#':
                    #print("Failed direction")
                    fail_flag = True
                    break
            dir_check += 1
            # If fail flag is false then we didn't fail!  
            if fail_flag == False:
                passed_elves.append((r1,c1,direction))
                add_flag = True
                
            if dir_check == 4:
                break
                  
    return passed_elves  
    
def move_calcs(elf_list, move_dict):
    """
    Take elves that can move, check for collisions and remove
    
    Important note: Don't need full matrix since we did a full 
    matrix check in the last step 
    
    """
    old_list = []
    new_list = []
    transition_list = []
    for (r,c, d) in elf_list:        
        # apply move
        #print(r,c,d)
        r_new = r + move_dict[d][0]
        c_new = c + move_dict[d][1]
        
        # store old and new locations for processing
        old_list.append((r,c))
        new_list.append((r_new, c_new))
        
    # checking for collisions: won't save collisions
    for idx, loc in enumerate(new_list):
        comp = [x for i, x in enumerate(new_list) if i != idx and x == loc]
        
        if len(comp) == 0:
            transition_list.append([old_list[idx], new_list[idx]])
    return transition_list

In [4]:
with open("data/day23_sample.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")
    
# get matrix starting dimensions
nrows = len(lines)
ncols = len(lines[0])
print(f"Starting shape: {nrows, ncols}")
    
# create matrix
matrix = np.asarray([[x for x in l] for l in lines])
matrix.shape = (nrows, ncols)

# each step we'd want to expand prior to checks
steps = 0 # always off by 1 from actual count
pad_size = 1
default = '.'

while steps <= 4:
    # expand border
    matrix = np.pad(matrix, pad_width=pad_size, mode='constant', constant_values=default)
    
    # figure out check order
    check_order = check_list[steps % len(check_list)]
    
    # find eligible elves for movement
    eligible_elves = full_scan(matrix, all_checks)
    
    if len(eligible_elves) == 0:
        print(f"No moves at {steps}")
        break
    
    # find potential moves
    moves = move_proposal(matrix, eligible_elves, check_dict, check_order)

    # output moves, accounting for collisions
    actual_movement = move_calcs(moves, move_dict)
    
    # make them move in the matrix
    for (old_move, new_move) in actual_movement:
        matrix[old_move] = '.'
        matrix[new_move] = '#'
    
    steps += 1

Starting shape: (6, 5)
No moves at 3


In [5]:
for row in range(matrix.shape[0]):
    print(''.join([x for x in matrix[row,:]]))

.............
.............
.............
.............
......#......
........#....
....#........
........#....
.............
......#......
.............
.............
.............
.............


In [6]:
# larger Sample
with open("data/day23_sample2.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")
    
# get matrix starting dimensions
nrows = len(lines)
ncols = len(lines[0])
print(f"Starting shape: {nrows, ncols}")
    
# create matrix
matrix = np.asarray([[x for x in l] for l in lines])
matrix.shape = (nrows, ncols)

# each step we'd want to expand prior to checks
steps = 0 # always off by 1 from actual count
pad_size = 1
default = '.'

while steps <= 9:
    print(f"On round {steps + 1}")
    # expand border
    matrix = np.pad(matrix, pad_width=pad_size, mode='constant', constant_values=default)
    
    # figure out check order
    check_order = check_list[steps % len(check_list)]
    
    # find eligible elves for movement
    eligible_elves = full_scan(matrix, all_checks)
    
    if len(eligible_elves) == 0:
        print(f"No moves at {step}")
        break
    
    # find potential moves
    moves = move_proposal(matrix, eligible_elves, check_dict, check_order)

    # output moves, accounting for collisions
    actual_movement = move_calcs(moves, move_dict)
    
    # make them move in the matrix
    for (old_move, new_move) in actual_movement:
        matrix[old_move] = '.'
        matrix[new_move] = '#'
    
    steps += 1

Starting shape: (12, 14)
On round 1
On round 2
On round 3
On round 4
On round 5
On round 6
On round 7
On round 8
On round 9
On round 10


In [7]:
for row in range(matrix.shape[0]):
    print(''.join([x for x in matrix[row,:]]))

..................................
..................................
..................................
..................................
..................................
..................................
..................................
..................................
..................................
..................................
.................#................
.....................#............
............#.#..#................
................#.................
.............#.....#..#...........
...........#......##..............
...............##.................
............#........#............
..............#.#..#..............
..................................
..............#..#..#.............
..................................
..................................
..................................
..................................
..................................
..................................
..................................
....................

In [8]:
# must find min row, max row, min col, max col
min_row = min([r[0] for r in np.argwhere(matrix == '#')])
max_row = max([r[0] for r in np.argwhere(matrix == '#')])
min_col = min([r[1] for r in np.argwhere(matrix == '#')])
max_col = max([r[1] for r in np.argwhere(matrix == '#')])

In [9]:
matrix[np.ix_([x for x in range(min_row, max_row + 1)], [x for x in range(min_col, max_col + 1)])]

array([['.', '.', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
       ['.', '#', '.', '#', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '#', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '#', '.', '.', '.', '.', '.', '#', '.', '.', '#'],
       ['#', '.', '.', '.', '.', '.', '.', '#', '#', '.', '.', '.'],
       ['.', '.', '.', '.', '#', '#', '.', '.', '.', '.', '.', '.'],
       ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
       ['.', '.', '.', '#', '.', '#', '.', '.', '#', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '#', '.', '.', '#', '.', '.', '#', '.', '.']],
      dtype='<U1')

In [10]:
final = matrix[np.ix_([x for x in range(min_row, max_row + 1)], [x for x in range(min_col, max_col + 1)])]
len(np.argwhere(final == '.'))

110

In [11]:
# larger Sample
with open("data/day23.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")
    
# get matrix starting dimensions
nrows = len(lines)
ncols = len(lines[0])
print(f"Starting shape: {nrows, ncols}")
    
# create matrix
matrix = np.asarray([[x for x in l] for l in lines])
matrix.shape = (nrows, ncols)

# each step we'd want to expand prior to checks
steps = 0 # always off by 1 from actual count
pad_size = 1
default = '.'

while steps <= 9:
    print(f"On round {steps + 1}")
    # expand border
    matrix = np.pad(matrix, pad_width=pad_size, mode='constant', constant_values=default)
    
    # figure out check order
    check_order = check_list[steps % len(check_list)]
    
    # find eligible elves for movement
    eligible_elves = full_scan(matrix, all_checks)
    
    if len(eligible_elves) == 0:
        print(f"No moves at {step}")
        break
    
    # find potential moves
    moves = move_proposal(matrix, eligible_elves, check_dict, check_order)

    # output moves, accounting for collisions
    actual_movement = move_calcs(moves, move_dict)
    
    # make them move in the matrix
    for (old_move, new_move) in actual_movement:
        matrix[old_move] = '.'
        matrix[new_move] = '#'
    
    steps += 1

Starting shape: (73, 73)
On round 1
On round 2
On round 3
On round 4
On round 5
On round 6
On round 7
On round 8
On round 9
On round 10


In [12]:
# must find min row, max row, min col, max col
min_row = min([r[0] for r in np.argwhere(matrix == '#')])
max_row = max([r[0] for r in np.argwhere(matrix == '#')])
min_col = min([r[1] for r in np.argwhere(matrix == '#')])
max_col = max([r[1] for r in np.argwhere(matrix == '#')])

final = matrix[np.ix_([x for x in range(min_row, max_row + 1)], [x for x in range(min_col, max_col + 1)])]
len(np.argwhere(final == '.'))

4049

In [13]:
# part 2: We just determine when we can't take anymore steps.
import time
with open("data/day23.txt", "r", encoding="UTF-8") as f:
    lines = f.read().split("\n")
    
# get matrix starting dimensions
nrows = len(lines)
ncols = len(lines[0])
print(f"Starting shape: {nrows, ncols}")
    
# create matrix
matrix = np.asarray([[x for x in l] for l in lines])
matrix.shape = (nrows, ncols)

# each step we'd want to expand prior to checks
steps = 0 # always off by 1 from actual count
pad_size = 1
default = '.'
s = time.time()
while True:
    #print(f"On round {steps + 1}")
    # expand border
    matrix = np.pad(matrix, pad_width=pad_size, mode='constant', constant_values=default)
    
    # figure out check order
    check_order = check_list[steps % len(check_list)]
    
    # find eligible elves for movement
    eligible_elves = full_scan(matrix, all_checks)
    
    if len(eligible_elves) == 0:
        print(f"No moves at {steps + 1}")
        print(f"Total time: {time.time() - s:.2f}")
        break
    
    # find potential moves
    moves = move_proposal(matrix, eligible_elves, check_dict, check_order)

    # output moves, accounting for collisions
    actual_movement = move_calcs(moves, move_dict)
    
    # make them move in the matrix
    for (old_move, new_move) in actual_movement:
        matrix[old_move] = '.'
        matrix[new_move] = '#'
    
    steps += 1

Starting shape: (73, 73)
No moves at 1021
Total time: 79.75
